In [ ]:

# Set up the notebook
%pprint
import sys
if (osp.join('..', 'py') not in sys.path): sys.path.insert(1, osp.join('..', 'py'))

In [ ]:

from FRVRS import (read_excel, to_datetime, Series)
import json

warnings.filterwarnings('ignore')

In [2]:

from pandas import 

file_path = '../data/xlsx/Alignment_x_Trust_Correlations.xlsx'
alignment_x_trust_correlations_df = read_excel(file_path)
alignment_x_trust_correlations_df.columns = alignment_x_trust_correlations_df.columns.map(
    lambda x: '_'.join([s.title() for s in x.split('_')])
)
alignment_x_trust_correlations_df.columns = alignment_x_trust_correlations_df.columns.map(lambda x: x[:2].upper() + x[2:])
alignment_x_trust_correlations_df = alignment_x_trust_correlations_df.rename(columns={'PArticipantid': 'participant_id'})
sorted(alignment_x_trust_correlations_df.columns)

['AD_Align_High_Four', 'AD_Align_High_Sim', 'AD_Align_High_Text', 'AD_Align_Low_Four', 'AD_Align_Low_Sim', 'AD_Align_Low_Text', 'AD_High_Agree', 'AD_High_Alignsr', 'AD_High_Trust', 'AD_High_Trustworthy', 'AD_Low_Agree', 'AD_Low_Alignsr', 'AD_Low_Trust', 'AD_Low_Trustworthy', 'ST_Align_High_Four', 'ST_Align_High_Sim', 'ST_Align_High_Text', 'ST_Align_Low_Four', 'ST_Align_Low_Sim', 'ST_Align_Low_Text', 'ST_High_Agree', 'ST_High_Agree_Omni', 'ST_High_Alignsr', 'ST_High_Alignsr_Omni', 'ST_High_Trust', 'ST_High_Trust_Omni', 'ST_High_Trustworthy', 'ST_High_Trustworthy_Omni', 'ST_Low_Agree', 'ST_Low_Agree_Omni', 'ST_Low_Alignsr', 'ST_Low_Alignsr_Omni', 'ST_Low_Trust', 'ST_Low_Trust_Omni', 'ST_Low_Trustworthy', 'ST_Low_Trustworthy_Omni', 'participant_id']

In [33]:

import pingouin as pg

def calculate_icc(participant_data, trust_rating_variable_name, alignment_score_variable_name, targets_column_name='participant_id'):
    """
    Calculates ICC(2,1) for a given Data Frame.
    
    Parameters:
        participant_data: The Data Frame containing participant data.
        trust_rating_variable_name: The column name for the trust rating variable.
        alignment_score_variable_name: The column name for the specific alignment score variable (e.g., AD_Align_High_Text).
        targets_column_name: The column name for participant ID. Defaults to 'participant_id'. Say you did measurements at n
            different settings, using k different methods, then 'targets' should look something like [1, 2, ..., n, 1, 2, ..., n],
            to indicate which rating belongs to which setting. Specifically, the function expects at least 5 non-missing
            values for each combination of target and rater.
    
    Returns:
        A pingouin ICC results object.
    """
    
    return pg.intraclass_corr(
        data=participant_data,
        targets=targets_column_name,
        raters=trust_rating_variable_name,
        ratings=alignment_score_variable_name,
        nan_policy='omit'
    )

In [32]:

calculate_icc(alignment_x_trust_correlations_df, 'AD_Align_High_Text', 'AD_High_Trust', targets_column_name='participant_id')

1

AssertionError: Data must have at least 5 non-missing values.


I have separate columns in the alignment_x_trust_correlations_dfn Data Frame for AD High Trust, AD Low Trust, etc. The relationships I want primarily need to understand are:

 - AD_Align_High_Text by AD_High_Trust
 - AD_Align_Low_Text by AD_Low_Trust

 - ST_Align_High_Text by ST_High_Trust
 - ST_Align_Low_Text by ST_Low_Trust

In [35]:

from scipy.stats import pointbiserialr

# Loop through desired groups
relationships_str = '''AD_Align_High_Text by AD_High_Trust|AD_Align_Low_Text by AD_Low_Trust
ST_Align_High_Text by ST_High_Trust|ST_Align_Low_Text by ST_Low_Trust'''

for relationship_str in relationships_str.split('\n'):
    high_low_list = relationship_str.split('|')
    high_variable_names_list = high_low_list[0].split(' by ')
    low_variable_names_list = high_low_list[1].split(' by ')
    
    # Define column names for high trust rating and high alignment score column name
    high_trust_rating_variable_name = high_variable_names_list[1]
    high_alignment_score_column_name = high_variable_names_list[0]
    
    # Select data for the high trust group (assuming separate columns)
    high_columns_list = ['participant_id', high_alignment_score_column_name, high_trust_rating_variable_name]
    high_trust_data = alignment_x_trust_correlations_df[high_columns_list].dropna()
    
    # Define column names for low trust rating and low alignment score column name
    low_trust_rating_variable_name = low_variable_names_list[1]
    low_alignment_score_column_name = low_variable_names_list[0]

    for trust_rating_variable_name in [
        high_trust_rating_variable_name, high_alignment_score_column_name, low_trust_rating_variable_name, low_alignment_score_column_name
    ]:
        columns_list = ['participant_id', trust_rating_variable_name]
        print(*columns_list, alignment_x_trust_correlations_df.groupby(columns_list).size().min())
    continue
    
    # Select data for the low trust group (assuming separate columns)
    low_columns_list = ['participant_id', low_alignment_score_column_name, low_trust_rating_variable_name]
    low_trust_data = alignment_x_trust_correlations_df[low_columns_list].dropna()
    
    # Calculate point-biserial correlation for high trust
    high_trust_corr, _ = pointbiserialr(high_trust_data[high_alignment_score_column_name], high_trust_data[high_trust_rating_variable_name])
    
    # Calculate point-biserial correlation for low trust
    low_trust_corr, _ = pointbiserialr(low_trust_data[low_alignment_score_column_name], low_trust_data[low_trust_rating_variable_name])
    
    # Print results for the current group
    print(f'Point-biserial Correlation ({high_low_list[0]}):')
    print(high_trust_corr)
    print(f'Point-biserial Correlation ({high_low_list[1]}):')
    print(low_trust_corr)

participant_id AD_High_Trust 1
participant_id AD_Align_High_Text 1
participant_id AD_Low_Trust 1
participant_id AD_Align_Low_Text 1
participant_id ST_High_Trust 1
participant_id ST_Align_High_Text 1
participant_id ST_Low_Trust 1
participant_id ST_Align_Low_Text 1



----

In [14]:

pg.read_dataset('icc').Wine.tolist()

[1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8]

In [16]:

alignment_x_trust_correlations_df.participant_id.tolist()

[2024201, 2024202, 2024203, 2024204, 2024205, 2024206, 2024207, 2024208, 2024209, 2024211, 2024212, 2024213, 2024214, 2024215, 2024216, 2024217, 2024218, 2024219, 2024220, 2024221, 2024222, 2024223, 2024224, 2024225, 2024226, 2024227]

In [27]:

new_sequence, string_to_integer_map = nu.convert_strings_to_integers(alignment_x_trust_correlations_df.participant_id.tolist())
new_sequence = [k+1 for k in new_sequence]
string_to_integer_map = {k: v+1 for k, v in string_to_integer_map.items()}
alignment_x_trust_correlations_df['new_sequence'] = new_sequence
calculate_icc(alignment_x_trust_correlations_df, 'AD_Align_High_Text', 'AD_High_Trust', targets_column_name='new_sequence')

AssertionError: Data must have at least 5 non-missing values.